In [100]:
 
import json

In [6]:
file="/Users/jaderazzaghi/Desktop/NLP/reviews.json"
with open(file)as f:
    for line in f:
        print(line)
        break 

 {"reviewerID": "A1E5ZR1Z4OQJG", "asin": "1495329321", "reviewerName": "Pure Jonel \"Pure Jonel\"", "helpful": [0, 0], "reviewText": "Da Silva takes the divine by storm with this unique new novel.  She develops a world unlike any others while keeping it firmly in the real world.  This is a very well written and entertaining novel.  I was quite impressed and intrigued by the way that this solid storyline was developed, bringing the readers right into the world of the story.  I was engaged throughout and definitely enjoyed my time spent reading it.I loved the character development in this novel.  Da Silva creates a cast of high school students who actually act like high school students.  I really appreciated the fact that none of them were thrown into situations far beyond their years, nor did they deal with events as if they had decades of life experience under their belts.  It was very refreshing and added to the realism and impact of the novel.  The friendships between the characters 

In [10]:
#we want the review text:
with open (file)as f:
    for line in f:
        review=json.loads(line)
        print(review['reviewText'])#just the review text part of json file
        print(review['overall'])#score of the review
        break 

Da Silva takes the divine by storm with this unique new novel.  She develops a world unlike any others while keeping it firmly in the real world.  This is a very well written and entertaining novel.  I was quite impressed and intrigued by the way that this solid storyline was developed, bringing the readers right into the world of the story.  I was engaged throughout and definitely enjoyed my time spent reading it.I loved the character development in this novel.  Da Silva creates a cast of high school students who actually act like high school students.  I really appreciated the fact that none of them were thrown into situations far beyond their years, nor did they deal with events as if they had decades of life experience under their belts.  It was very refreshing and added to the realism and impact of the novel.  The friendships between the characters in this novel were also truly touching.Overall, this novel was fantastic.  I can&#8217;t wait to read more and to find out what happen

In [12]:
#so we just want text and score so to merge them:
reviews=[]
with open(file)as f:
    for lines in f:
        review=json.loads(lines)
        reviews.append((review['reviewText'],review['overall']))

In [16]:
#check if we created a good file:
print(reviews[5])
print(reviews[5][1])

('Love the book, great story line, keeps you entertained.for a first novel from this author she did a great job,  Would definitely recommend!', 4.0)
4.0


In [28]:
# to make it neat we create a class:
class Sentiment:
    neg='NEGATIVE'
    pos='POSITIVE'
    neu='NEUTRAL'
class Review:
    def __init__(self, text, score):
        self.text=text
        self.score=score
        self.sentiment=self.get_sentiment()
    def get_sentiment(self):
        if self.score<=2:
            return Sentiment.neg
        elif self.score==3:
            return Sentiment.neu
        else:
            return Sentiment.pos

In [31]:
reviews=[]
with open(file)as f:
    for lines in f:
        review=json.loads(lines)
        reviews.append(Review(review['reviewText'],review['overall']))
        
        
print(reviews[5].score,'\n')
print(reviews[1].text,'\n')
print(reviews[1].sentiment)

4.0 

For me personally it's the most disappointing book I have ready re: Kay Scarpetta. I generally can't put a Cornwell book down, but I had to force myself to keep reading it. Hard to stay focused when half a book describes approximately 4-6 hours. 

NEGATIVE


In [32]:
len(reviews)

1000

In [40]:
#split our data:
from sklearn.model_selection import train_test_split
training,test=train_test_split(reviews, test_size=0.33, random_state=42)
len(test)

330

In [43]:
print(training[1].sentiment)
print(training[1].text)

POSITIVE
Oh this is an awful bookA delightful awful bookJust the sort of awful bookYour whole family will adoreMy daughters requested it so often from the library, I finally just bought a copy.


In [49]:
#now we split the whole file we need to have our train and test (including text for x and labels for y)
train_X=[x.text for x in training]
train_y=[x.sentiment for x in training]
test_X=[x.text for x in test]
test_y=[x.sentiment for x in test]
train_y[0]
test_X[2]

'Michael Cunningham mesmerizes with the thoughtful, elegant prose that is this book.  The reader becomes so close to its characters...the reader feels what these brothers feel.  Beautiful and tragic...a book that will stay with me for a long, long time.  Thank you again, Mr. Cunningham.  The Hours remains at the top of my list and The Snow Queen is another gift to your readers.'

In [72]:
#now turn the train and test into vectors using BOW:
from sklearn.feature_extraction.text import CountVectorizer
cv=CountVectorizer()
train_xvector=cv.fit_transform(train_X)
test_xvector=cv.transform(test_X)

In [73]:
print(train_X[0])
print(train_xvector[0]. toarray())

Vivid characters and descriptions. The author has created a tale that grabs your attention and I couldn't put it down.
[[0 0 0 ... 0 0 0]]


In [74]:
#traing our model:
from sklearn import svm
from sklearn.svm import LinearSVC
clf_svm=svm.SVC(kernel='linear')
clf_svm.fit(train_xvector, train_y)

SVC(kernel='linear')

In [75]:
#lets do prediction
test_X[0]
clf_svm.predict(test_xvector[0])

array(['POSITIVE'], dtype='<U8')

In [78]:
#now try with other models:
#1Decision Tree:
from sklearn.tree import DecisionTreeClassifier
clf_DT=DecisionTreeClassifier()
clf_DT.fit(train_xvector, train_y)

DecisionTreeClassifier()

In [79]:
clf_DT.predict(test_xvector[0])

array(['POSITIVE'], dtype='<U8')

In [ ]:
#from sklearn.naive_bayes import GaussianNB
#clf_NB=GaussianNB()
#clf_NB.fit(train_xvector, train_y)

In [84]:
from sklearn.linear_model import LogisticRegression
clf_log=LogisticRegression()
clf_log.fit(train_xvector, train_y)
clf_log.predict(test_xvector[0])

/opt/anaconda3/lib/python3.8/site-packages/sklearn/linear_model/_logistic.py:762: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


array(['POSITIVE'], dtype='<U8')

In [90]:
#let's evaluate the models:This is mean accuracy score
from sklearn.metrics import confusion_matrix, classification_report, accuracy_score
print(clf_svm.score(test_xvector, test_y))
print(clf_DT.score(test_xvector, test_y))
print(clf_log.score(test_xvector, test_y))

0.8242424242424242
0.7636363636363637
0.8303030303030303


In [96]:
#Measure F1score:
from sklearn.metrics import f1_score
f1_score(test_y, clf_svm.predict(test_xvector), average=None, labels=[Sentiment.pos, Sentiment.neg, Sentiment.neu])
#so really hi 91 for pos, but trashy for the neg and neu

array([0.91319444, 0.22222222, 0.21052632])

In [97]:
f1_score(test_y, clf_DT.predict(test_xvector), average=None,labels=[Sentiment.pos, Sentiment.neg, Sentiment.neu])

array([0.86865149, 0.06896552, 0.1       ])

In [98]:
f1_score(test_y, clf_log.predict(test_xvector), average=None,labels=[Sentiment.pos, Sentiment.neg, Sentiment.neu])

array([0.91370558, 0.1       , 0.12244898])

In [99]:
#so now we need to fix this issue:
train_y.count(Sentiment.pos)# so we have alot of pos in our y train so we need to balance our pos&neg data

552